In [1]:
import seaborn as sns
import pandas as pnd
import matplotlib.pyplot as plt
import os
import random as rnd
import statistics as stat
import numpy as np

# Model

Will ich das ganze hier Objektorientiert angehen? Ich könnte jeden Akteur als Objekt betrachten, mit assesment und peers. Ich frage mich aber auch schon, ob das Programm dann schnell läuft, oder ob es sich noch irgendwie optimieren lässt.

Große Frage: Wie programmiere ich die Zeitschritte optimal?

Ich will:

- update the assesment of every agent to their observation and the mean of their peers
- update the peers of every agent

Dabei frage ich mich, ob ich die peers von jedem agent schneller finden kann als, indem ich jedes mal jeden anderen Akteur betrachte. (Das ist wichtig, weil die Simulationen sonst lange dauern)

Das ginge, wenn ich die Akteure in einer Liste speichere, die nach ihrem assesment sortiert ist. Dann muss ich mich nur vom gegenwärtigen Akteur nach links und rechts vortasten und kann aufhören, sobald ich zwei finde, die weiter weg sind als epsilon. Ich müsste also die Liste sortieren, nachdem ich die assesments geupdated habe. Könnte sein, dass ich dabei ein paar Schritte spare.

Ich programmiere es jetzt erstmal brute force und wenn ich Geschwindigkeitsprobleme kriege, probier ich die Version.

In [95]:
# --- globals --- #

# let there be n agents with random initial assesements
# every agent is represented by an integer in [0, n)
n = 10
epsilon = .3
alpha = .4
noise = .5
tau = .1
max_time = 5

In [90]:
# --- classes --- #

class Agent:
    def __init__(self, id):
        self.assesment = 1 - rnd.random() # values in (0, 1]
        self.peers = set()
        self.id = id
    
    def __str__(self):
        return f"assesment: {self.assesment} and peers: {self.peers}"

    def __repr__(self):
        return f"(Agent [{self.id}]: {self.assesment:.2f}, {self.peers})"

    def update_peers(self, agent_list):
        for potential_peer in agent_list:
            if (potential_peer.id != self.id and 
                    abs(potential_peer.assesment - self.assesment < epsilon)):
                self.peers.add(potential_peer)

In [91]:
# --- functions --- #

def print_agent_list(agent_list):
    print("[")
    for agent in agent_list:
        representation = f"    (Agent [{agent.id}]: {agent.assesment:.2f}, " + "{"
        representation += ", ".join(f"[{peer.id}]" for peer in agent.peers) + "})"
    
        print(representation)
    print("]")

In [92]:
# --- main code --- #

agents = [Agent(i) for i in range(n)]
print_agent_list(agents)

[
    (Agent [0]: 0.47, {})
    (Agent [1]: 0.17, {})
    (Agent [2]: 0.85, {})
    (Agent [3]: 0.12, {})
    (Agent [4]: 0.34, {})
    (Agent [5]: 0.92, {})
    (Agent [6]: 0.98, {})
    (Agent [7]: 0.30, {})
    (Agent [8]: 0.02, {})
    (Agent [9]: 0.64, {})
]


In [93]:
# for each agent let there be a set of agents with similar assesments
for i in range(n):
    agents[i].update_peers(agents)

print_agent_list(agents)

[
    (Agent [0]: 0.47, {[7], [9], [3], [8], [4], [1]})
    (Agent [1]: 0.17, {[7], [3], [8], [0], [4]})
    (Agent [2]: 0.85, {[7], [5], [9], [3], [8], [0], [6], [4], [1]})
    (Agent [3]: 0.12, {[7], [4], [1], [8]})
    (Agent [4]: 0.34, {[7], [3], [8], [0], [1]})
    (Agent [5]: 0.92, {[7], [2], [9], [3], [8], [0], [6], [4], [1]})
    (Agent [6]: 0.98, {[7], [2], [5], [9], [3], [8], [0], [4], [1]})
    (Agent [7]: 0.30, {[3], [8], [0], [4], [1]})
    (Agent [8]: 0.02, {[7], [1], [3]})
    (Agent [9]: 0.64, {[7], [2], [5], [3], [8], [0], [4], [1]})
]


In [96]:
# for each time step
for u in range(max_time):
    # update the assesment of each agent to a ratio between the agents observations and the mean of their peers
    for agent in agents:
        # observing
        observation = np.random.normal(tau, noise/2) # random value from bell curve with tau as 2 std deviations
        while observation <= 0 or observation > 1: # cutting off values outside (0, 1]
            observation = np.random.normal(tau, noise/2)
        
        if agent.peers:
            observation *= (1- alpha)

            # listening to peers
            collective_peer_assesment = alpha * stat.mean([peer.assesment for peer in agent.peers])

            agent.assesment = observation + collective_peer_assesment
        else:
            agent.assesment = observation
    
    # update each agents peers
    for agent in agents:
        agent.update_peers(agents)
    
    print_agent_list(agents)

        

[
    (Agent [0]: 0.08, {[7], [2], [5], [9], [3], [8], [6], [4], [1]})
    (Agent [1]: 0.26, {[7], [2], [5], [9], [3], [8], [0], [6], [4]})
    (Agent [2]: 0.28, {[7], [5], [9], [3], [8], [0], [6], [4], [1]})
    (Agent [3]: 0.14, {[7], [2], [5], [9], [8], [0], [6], [4], [1]})
    (Agent [4]: 0.24, {[7], [5], [9], [3], [8], [0], [6], [2], [1]})
    (Agent [5]: 0.10, {[7], [2], [9], [3], [8], [0], [6], [4], [1]})
    (Agent [6]: 0.16, {[7], [2], [5], [9], [3], [8], [0], [4], [1]})
    (Agent [7]: 0.29, {[2], [5], [9], [3], [8], [0], [6], [4], [1]})
    (Agent [8]: 0.08, {[7], [5], [9], [4], [3], [0], [6], [2], [1]})
    (Agent [9]: 0.14, {[7], [2], [5], [3], [8], [0], [6], [4], [1]})
]
[
    (Agent [0]: 0.32, {[7], [2], [5], [9], [3], [8], [6], [4], [1]})
    (Agent [1]: 0.35, {[7], [2], [5], [9], [3], [8], [0], [6], [4]})
    (Agent [2]: 0.10, {[7], [5], [9], [3], [8], [0], [6], [4], [1]})
    (Agent [3]: 0.19, {[7], [2], [5], [9], [8], [0], [6], [4], [1]})
    (Agent [4]: 0.27, {[7], 